# Data Preprocessing

In [10]:
# Import required libraries
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from datetime import datetime, date, time
import requests
import pandas as pd
import numpy as np
import xlrd
import asyncio
import aiohttp
from time import sleep
from statistics import mean

Firstly, we need to identify the relevant data files from the dataset and define which columns in our data are relevant for the analysis.



In [11]:
datasets_list = []

# Read and save each dataset(they have different extensions)
for i in range(22):
    if (i < 13):
        exec("df" + str(i) + " = pd.read_excel('atp_20" + (("0" + str(i))[-2:]) + ".xls')")
    else:
        exec("df" + str(i) + " = pd.read_excel('atp_20" + (("0" + str(i))[-2:]) + ".xlsx')")
        
    exec("datasets_list.append(df" + str(i) + ")")

C:\Users\markz\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\markz\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\markz\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\markz\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [12]:
# Change order of columns 17 datasets
for i in range(5, 22):
    exec("col = df" + str(i) + ".pop('WPts')")
    exec("df" + str(i) + ".insert(" + "df" + str(i) + ".shape[1], 'WPts', col)")
    exec("col = df" + str(i) + ".pop('LPts')")
    exec("df" + str(i) + ".insert(" + "df" + str(i) + ".shape[1], 'LPts', col)")

Initially, we had 22 atp games datasets with varying numbers of columns. To streamline our analysis, we decided to only use the columns present in the first dataset. This was because the other datasets often contained extraneous information such as bookmakers' coefficients and other data not relevant to our analysis. To consolidate the data, we concatenated all 22 datasets into one large dataset and worked with that and atp_players.csv file.

In [19]:
# Concatenate the datasets vertically and leave useful rows only
merged_df = pd.concat(datasets_list, ignore_index=True)
merged_df = merged_df.loc[:, 'ATP':'Comment'].query('Comment == "Completed" and W2.notnull() and LRank.notnull() and WRank.notnull() and LRank != "NR"')

In [21]:
merged_df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1',
       'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets',
       'Comment'],
      dtype='object')

In [9]:
merged_df.columns

Index(['Location', 'Date', 'Series', 'Court', 'Surface', 'Winner', 'Loser',
       'Wsets_percent', 'W_round_pts_avg_percent', 'Month', 'Day', 'Year',
       'W_over_L_rank_difference_normalized'],
      dtype='object')

In [22]:
# First half of atp games dataset
merged_df.loc[:, 'ATP':'Winner'].head(2)

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner
0,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.
1,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.


In [24]:
# Second half of atp games dataset
merged_df.loc[:, 'Winner':'Comment'].head(2)

,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment
0,Dosedel S.,Ljubicic I.,63,77,6.0,4.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed
1,Enqvist T.,Clement A.,5,56,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed


Since there is a significant amount of scores and sets information available, it may be beneficial to condense the data in order to highlight the most relevant information.

In [6]:
# Drop ATP, Tournament, Round, Comment columns, because its useless for our analysis
merged_df.drop(merged_df.columns[[0, 2, 7, 25]], axis=1, inplace=True)

# Get sets percentage of winner player
merged_df["Wsets_percent"] = merged_df["Wsets"]/(merged_df["Wsets"] + merged_df["Lsets"])

# Fill null values in short games
merged_df['W3'] = merged_df['W3'].replace(' ', 0)
merged_df['L3'] = merged_df['L3'].replace(' ', 0)
merged_df = merged_df.fillna(0)

# Make columns for average number of pts per round for each player and leaving only the column with winner average number of pts per round percent
merged_df['W_round_pts_avg'] = (merged_df["W1"] + merged_df["W2"] + merged_df["W3"] + merged_df["W4"] + merged_df["W5"])/(merged_df["Wsets"] + merged_df["Lsets"])
merged_df['L_round_pts_avg'] = (merged_df["L1"] + merged_df["L2"] + merged_df["L3"] + merged_df["L4"] + merged_df["L5"])/(merged_df["Wsets"] + merged_df["Lsets"])
merged_df['W_round_pts_avg_percent'] = merged_df['W_round_pts_avg']/(merged_df['W_round_pts_avg'] + merged_df['L_round_pts_avg'])

# Extract the month, day, and year from the 'Date' column
merged_df['Month'] = merged_df['Date'].dt.month
merged_df['Day'] = merged_df['Date'].dt.day
merged_df['Year'] = merged_df['Date'].dt.year

# Make normalized rand difference between winner and loser
merged_df['W_over_L_rank_difference_normalized'] = np.log(merged_df['LRank']/merged_df['WRank'])

# Now we get useful information from some column, but now we don't need them, so drop it
merged_df.drop(merged_df.columns[[5, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24]], axis=1, inplace=True)

By analyzing the dates associated with our player's, we can determine whether or not they have previously visited a given city.

In [56]:
dates_df = merged_df[['Location', 'Winner', 'Loser', 'Date']].copy()
dates_df = dates_df.melt(id_vars=['Location', 'Date'], value_vars=['Winner', 'Loser'], var_name='Player Type', value_name='Player')
dates_df = dates_df.groupby(['Location', 'Player']).agg({'Date': 'min'}).reset_index()
dates_df = dates_df.rename(columns={'Date': 'Player_min_date'})
merged_df = pd.merge(merged_df, dates_df, left_on=['Location', 'Winner'], right_on=['Location', 'Player'], how='left')
merged_df = pd.merge(merged_df, dates_df, left_on=['Location', 'Loser'], right_on=['Location', 'Player'], how='left')

merged_df['W_played_here_before'] = merged_df['Date'] > merged_df['Player_min_date_x']
merged_df['L_played_here_before'] = merged_df['Date'] > merged_df['Player_min_date_y']

merged_df = merged_df.drop(columns=['Player_min_date_x', 'Player_min_date_y', 'Player_x', 'Player_y'])
merged_df

,Location,Date,Series,Court,Surface,Winner,Loser,Wsets_percent,W_round_pts_avg_percent,Month,Day,Year,W_over_L_rank_difference_normalized,W_played_here_before,L_played_here_before
0,Adelaide,2000-01-03,International,Outdoor,Hard,Dosedel S.,Ljubicic I.,1.000000,0.666667,1,3,2000,0.200671,False,False
1,Adelaide,2000-01-03,International,Outdoor,Hard,Enqvist T.,Clement A.,1.000000,0.666667,1,3,2000,2.415914,False,False
2,Adelaide,2000-01-03,International,Outdoor,Hard,Escude N.,Baccanello P.,0.666667,0.558824,1,3,2000,2.795756,False,False
3,Adelaide,2000-01-03,International,Outdoor,Hard,Federer R.,Knippschild J.,1.000000,0.705882,1,3,2000,0.291521,False,False
4,Adelaide,2000-01-03,International,Outdoor,Hard,Fromberg R.,Woodbridge T.,0.666667,0.514286,1,3,2000,0.893818,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56379,Turin,2021-11-19,Masters Cup,Indoor,Hard,Ruud C.,Rublev A.,0.666667,0.484848,11,19,2021,-0.470004,True,True
56380,Turin,2021-11-19,Masters Cup,Indoor,Hard,Djokovic N.,Norrie C.,1.000000,0.800000,11,19,2021,2.484907,True,True
56381,Turin,2021-11-20,Masters Cup,Indoor,Hard,Medvedev D.,Ruud C.,1.000000,0.666667,11,20,2021,1.386294,True,True
56382,Turin,2021-11-20,Masters Cup,Indoor,Hard,Zverev A.,Djokovic N.,0.666667,0.531250,11,20,2021,-1.098612,True,True


To enhance our analysis, we could consider incorporating external weather data for the locations and dates. This would allow us to investigate potential relationships between weather conditions and our player's performance.

In [39]:
# For Indoor games weather doesn't matter, but we can work with them later
notweather_dataset = merged_df.query('Court == "Indoor"').loc[:, : 'Date'].drop_duplicates()

# Now we need to get latitude and longitude for every unique city
weather_dataset = merged_df.query('Court == "Outdoor"').loc[:, : 'Date'].drop_duplicates()
locations = list(merged_df['Location'].drop_duplicates().reset_index(drop=True))
latitude = [requests.get(f"https://nominatim.openstreetmap.org/search?q={locations[i]}&format=json").json()[0]["lat"] for i in range(len(locations))]
longitude = [requests.get(f"https://nominatim.openstreetmap.org/search?q={locations[i]}&format=json").json()[0]["lon"] for i in range(len(locations))]

# Сreate dataset that includes the Location, Date, Latitude, and Longitude
data_tuples = list(zip(locations, latitude, longitude))
locations_dataset = pd.DataFrame(data_tuples, columns=['Location', 'Latitude', 'Longitude'])
weather_dataset = pd.merge(weather_dataset, locations_dataset, on='Location')

weather_dataset

,Location,Date,Latitude,Longitude
0,Adelaide,2000-01-03,-34.9281805,138.5999312
1,Adelaide,2001-01-01,-34.9281805,138.5999312
2,Adelaide,2001-12-31,-34.9281805,138.5999312
3,Adelaide,2002-12-30,-34.9281805,138.5999312
4,Adelaide,2002-12-31,-34.9281805,138.5999312
...,...,...,...,...
6984,San Diego,2021-09-30,32.7174202,-117.1627728
6985,San Diego,2021-10-01,32.7174202,-117.1627728
6986,San Diego,2021-10-02,32.7174202,-117.1627728
6987,San Diego,2021-10-03,32.7174202,-117.1627728


To collect weather data for each Location/Date pair in our dataset, we will need to make 6989 asynchronous requests.

In [40]:
async def get_weather_info(session, latitude, longitude, game_date):
    API = f'https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={game_date}&end_date={game_date}&hourly=temperature_2m,windspeed_10m,relativehumidity_2m,surface_pressure,precipitation,cloudcover,shortwave_radiation'
    async with session.get(API) as response:
        return await response.json()

latitude = list(weather_dataset['Latitude'])
longitude = list(weather_dataset['Longitude'])
dates = list(weather_dataset['Date'])
weather_info = []

formatted_dates = []
for date in weather_dataset['Date']:
    timestamp = pd.Timestamp(date)
    formatted_dates.append(timestamp.strftime('%Y-%m-%d'))

async with aiohttp.ClientSession() as session:
    tasks = []
    for j in range(14):
        if j != 13:
            for i in range(500):
                task = asyncio.create_task(get_weather_info(session, weather_dataset['Latitude'][j*500 + i], weather_dataset['Longitude'][j*500 + i], formatted_dates[j*500 + i]))
                tasks.append(task)
        else:
            for i in range(489):
                task = asyncio.create_task(get_weather_info(session, weather_dataset['Latitude'][j*500 + i], weather_dataset['Longitude'][j*500 + i], formatted_dates[j*500 + i]))
                tasks.append(task)
        await asyncio.sleep(3)

    for response in await asyncio.gather(*tasks):
        weather_info.append(response)

In [41]:
temperature = []
windspeed_10m = []
relativehumidity_2m = []
surface_pressure = []
precipitation = []
cloudcover = []
shortwave_radiation = []

# Save weather info in lists
for i in range(len(weather_info)):
    temperature.append(mean(weather_info[i]['hourly']['temperature_2m'][11:17]))
    windspeed_10m.append(mean(weather_info[i]['hourly']['windspeed_10m'][11:17]))    
    relativehumidity_2m.append(mean(weather_info[i]['hourly']['relativehumidity_2m'][11:17]))
    surface_pressure.append(mean(weather_info[i]['hourly']['surface_pressure'][11:17]))
    precipitation.append(mean(weather_info[i]['hourly']['precipitation'][11:17]))
    cloudcover.append(mean(weather_info[i]['hourly']['cloudcover'][11:17]))
    shortwave_radiation.append(mean(weather_info[i]['hourly']['shortwave_radiation'][11:17]))

In [42]:
# Great! We now have access to the weather dataset for our outdoor games
weather_dataset.drop(weather_dataset.columns[[2,3]], axis=1, inplace=True)
weather_dataset['Temperature'] = temperature
weather_dataset['Windspeed_10m'] = windspeed_10m
weather_dataset['Relativehumidity_2m'] = relativehumidity_2m
weather_dataset['Surface_pressure'] = surface_pressure
weather_dataset['Precipitation'] = precipitation
weather_dataset['Сloudcover'] =cloudcover
weather_dataset['Shortwave_radiation'] = shortwave_radiation
weather_dataset

,Location,Date,Temperature,Windspeed_10m,Relativehumidity_2m,Surface_pressure,Precipitation,Сloudcover,Shortwave_radiation
0,Adelaide,2000-01-03,15.050000,27.750000,68.000000,1004.016667,0.000000,40.500000,0.000000
1,Adelaide,2001-01-01,30.633333,27.366667,23.333333,997.966667,0.000000,17.000000,0.000000
2,Adelaide,2001-12-31,15.533333,31.366667,72.500000,992.166667,0.016667,50.500000,0.000000
3,Adelaide,2002-12-30,18.716667,27.333333,85.500000,989.450000,0.633333,68.000000,0.000000
4,Adelaide,2002-12-31,13.233333,27.933333,85.500000,997.083333,0.183333,48.500000,0.000000
...,...,...,...,...,...,...,...,...,...
6984,San Diego,2021-09-30,16.633333,2.666667,91.500000,1010.066667,0.000000,19.333333,76.500000
6985,San Diego,2021-10-01,18.950000,1.883333,76.166667,1010.783333,0.000000,0.000000,76.000000
6986,San Diego,2021-10-02,17.816667,3.866667,66.166667,1011.483333,0.000000,18.000000,70.166667
6987,San Diego,2021-10-03,17.816667,3.250000,66.833333,1010.300000,0.000000,11.000000,63.833333


Since indoor games are not affected by weather conditions, we can assume that the playing conditions are ideal for those games.

In [43]:
notweather_dataset = notweather_dataset.reset_index(drop=True)
temperature = [23 for i in range(2190)]
shortwave_radiation_precipitation_windspeed_10m = [0 for i in range(2190)]
relativehumidity = [50 for i in range(2190)]
surface_pressure = [1013.25 for i in range(2190)]
сloudcover = [100 for i in range(2190)]

notweather_dataset['Temperature'] = temperature
notweather_dataset['Windspeed_10m'] = shortwave_radiation_precipitation_windspeed_10m
notweather_dataset['Relativehumidity_2m'] = relativehumidity
notweather_dataset['Surface_pressure'] = surface_pressure
notweather_dataset['Precipitation'] = shortwave_radiation_precipitation_windspeed_10m
notweather_dataset['Сloudcover'] = сloudcover
notweather_dataset['Shortwave_radiation'] = shortwave_radiation_precipitation_windspeed_10m
notweather_dataset

,Location,Date,Temperature,Windspeed_10m,Relativehumidity_2m,Surface_pressure,Precipitation,Сloudcover,Shortwave_radiation
0,Marseille,2000-02-07,23,0,50,1013.25,0,100,0
1,San Jose,2000-02-07,23,0,50,1013.25,0,100,0
2,Memphis,2000-02-14,23,0,50,1013.25,0,100,0
3,Rotterdam,2000-02-14,23,0,50,1013.25,0,100,0
4,London,2000-02-21,23,0,50,1013.25,0,100,0
...,...,...,...,...,...,...,...,...,...
2185,Turin,2021-11-17,23,0,50,1013.25,0,100,0
2186,Turin,2021-11-18,23,0,50,1013.25,0,100,0
2187,Turin,2021-11-19,23,0,50,1013.25,0,100,0
2188,Turin,2021-11-20,23,0,50,1013.25,0,100,0


In [44]:
# Merge datasets for weather
full_weather_dataset = pd.concat([weather_dataset, notweather_dataset], ignore_index=True)
full_weather_dataset

,Location,Date,Temperature,Windspeed_10m,Relativehumidity_2m,Surface_pressure,Precipitation,Сloudcover,Shortwave_radiation
0,Adelaide,2000-01-03,15.050000,27.750000,68.000000,1004.016667,0.000000,40.5,0.0
1,Adelaide,2001-01-01,30.633333,27.366667,23.333333,997.966667,0.000000,17.0,0.0
2,Adelaide,2001-12-31,15.533333,31.366667,72.500000,992.166667,0.016667,50.5,0.0
3,Adelaide,2002-12-30,18.716667,27.333333,85.500000,989.450000,0.633333,68.0,0.0
4,Adelaide,2002-12-31,13.233333,27.933333,85.500000,997.083333,0.183333,48.5,0.0
...,...,...,...,...,...,...,...,...,...
9174,Turin,2021-11-17,23.000000,0.000000,50.000000,1013.250000,0.000000,100.0,0.0
9175,Turin,2021-11-18,23.000000,0.000000,50.000000,1013.250000,0.000000,100.0,0.0
9176,Turin,2021-11-19,23.000000,0.000000,50.000000,1013.250000,0.000000,100.0,0.0
9177,Turin,2021-11-20,23.000000,0.000000,50.000000,1013.250000,0.000000,100.0,0.0


In [58]:
# Merge weather information with main dataset
merged_df = pd.merge(merged_df, full_weather_dataset, on=['Location', 'Date'])
merged_df.drop(merged_df.columns[[0]], axis=1, inplace=True)
merged_df

,Date,Series,Court,Surface,Winner,Loser,Wsets_percent,W_round_pts_avg_percent,Month,Day,...,W_over_L_rank_difference_normalized,W_played_here_before,L_played_here_before,Temperature,Windspeed_10m,Relativehumidity_2m,Surface_pressure,Precipitation,Сloudcover,Shortwave_radiation
0,2000-01-03,International,Outdoor,Hard,Dosedel S.,Ljubicic I.,1.000000,0.666667,1,3,...,0.200671,False,False,15.05,27.75,68.0,1004.016667,0.0,40.5,0.0
1,2000-01-03,International,Outdoor,Hard,Enqvist T.,Clement A.,1.000000,0.666667,1,3,...,2.415914,False,False,15.05,27.75,68.0,1004.016667,0.0,40.5,0.0
2,2000-01-03,International,Outdoor,Hard,Escude N.,Baccanello P.,0.666667,0.558824,1,3,...,2.795756,False,False,15.05,27.75,68.0,1004.016667,0.0,40.5,0.0
3,2000-01-03,International,Outdoor,Hard,Federer R.,Knippschild J.,1.000000,0.705882,1,3,...,0.291521,False,False,15.05,27.75,68.0,1004.016667,0.0,40.5,0.0
4,2000-01-03,International,Outdoor,Hard,Fromberg R.,Woodbridge T.,0.666667,0.514286,1,3,...,0.893818,False,False,15.05,27.75,68.0,1004.016667,0.0,40.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56379,2021-11-19,Masters Cup,Indoor,Hard,Ruud C.,Rublev A.,0.666667,0.484848,11,19,...,-0.470004,True,True,23.00,0.00,50.0,1013.250000,0.0,100.0,0.0
56380,2021-11-19,Masters Cup,Indoor,Hard,Djokovic N.,Norrie C.,1.000000,0.800000,11,19,...,2.484907,True,True,23.00,0.00,50.0,1013.250000,0.0,100.0,0.0
56381,2021-11-20,Masters Cup,Indoor,Hard,Medvedev D.,Ruud C.,1.000000,0.666667,11,20,...,1.386294,True,True,23.00,0.00,50.0,1013.250000,0.0,100.0,0.0
56382,2021-11-20,Masters Cup,Indoor,Hard,Zverev A.,Djokovic N.,0.666667,0.531250,11,20,...,-1.098612,True,True,23.00,0.00,50.0,1013.250000,0.0,100.0,0.0


We will need to work with the atp players dataset.

In [22]:
players_df = pd.read_csv('atp_players.csv')

# Drop useless and corrupted columns(a lot of null values)
players_df.drop(players_df.columns[[0, 1, 2, 3, 5, 7, 8, 9, 11, 12, 13, 14, 16, 17, 19, 20]], axis=1, inplace=True)

# Fill other columns with null values with medians
players_df['birth_year'] = players_df['birth_year'].fillna(players_df['birth_year'].median())
players_df['weight_kg'] = players_df['weight_kg'].fillna(players_df['weight_kg'].median())
players_df['weight_kg'].replace(0, players_df['weight_kg'].median(), inplace=True)
players_df['height_cm'] = players_df['height_cm'].fillna(players_df['height_cm'].median())
players_df['height_cm'].replace(0, players_df['height_cm'].median(), inplace=True)

# Fill flag column with unknown flag with 'UNK' values
players_df['flag_code'] = players_df['flag_code'].fillna('UNK')
players_df['full_name'] = players_df['full_name'].apply(lambda x: x + '.')
players_df

,full_name,flag_code,birth_year,weight_kg,height_cm
0,Acuna R.,CHI,1958.0,68.0,175.0
1,Abdullahi S.,NGR,1960.0,77.0,183.0
2,Aerts N.,BRA,1963.0,75.0,188.0
3,Adams E.,USA,1959.0,73.0,178.0
4,Agenor R.,USA,1964.0,82.0,180.0
...,...,...,...,...,...
10907,Zorab A.,UNK,1976.0,77.0,183.0
10908,Zimmermann J.,UNK,1976.0,77.0,183.0
10909,Zinn G.,UNK,1976.0,77.0,183.0
10910,Zenger C.,GER,1976.0,77.0,183.0


In [18]:
players_df[players_df.duplicated(keep=False)]

,full_name,flag_code,birth_year,weight_kg,height_cm
60,Unknown U.,UNK,1976.0,77.0,183.0
81,Andersson K.,SWE,1976.0,77.0,183.0
132,Andersson K.,SWE,1976.0,77.0,183.0
165,Aubone R.,ARG,1976.0,77.0,183.0
170,Aubone R.,ARG,1976.0,77.0,183.0
...,...,...,...,...,...
10609,Weber J.,UNK,1976.0,77.0,183.0
10620,Weber J.,UNK,1976.0,77.0,183.0
10622,Weber J.,UNK,1976.0,77.0,183.0
10645,Watson C.,UNK,1976.0,77.0,183.0


In [23]:
# Change names for more suitable
winner_players_df = players_df.copy()
winner_players_df = winner_players_df.rename(columns={'full_name': 'Winner', 'flag_code': 'W_flag_code', 'birth_year': 'W_birth_year', 'weight_kg': 'W_weight_kg', 'height_cm': 'W_height_cm'})

loser_players_df = players_df.copy()
loser_players_df = loser_players_df.rename(columns={'full_name': 'Loser', 'flag_code': 'L_flag_code', 'birth_year': 'L_birth_year', 'weight_kg': 'L_weight_kg', 'height_cm': 'L_height_cm'})

In [24]:
winner_players_df

,Winner,W_flag_code,W_birth_year,W_weight_kg,W_height_cm
0,Acuna R.,CHI,1958.0,68.0,175.0
1,Abdullahi S.,NGR,1960.0,77.0,183.0
2,Aerts N.,BRA,1963.0,75.0,188.0
3,Adams E.,USA,1959.0,73.0,178.0
4,Agenor R.,USA,1964.0,82.0,180.0
...,...,...,...,...,...
10907,Zorab A.,UNK,1976.0,77.0,183.0
10908,Zimmermann J.,UNK,1976.0,77.0,183.0
10909,Zinn G.,UNK,1976.0,77.0,183.0
10910,Zenger C.,GER,1976.0,77.0,183.0


In [25]:
loser_players_df

,Loser,L_flag_code,L_birth_year,L_weight_kg,L_height_cm
0,Acuna R.,CHI,1958.0,68.0,175.0
1,Abdullahi S.,NGR,1960.0,77.0,183.0
2,Aerts N.,BRA,1963.0,75.0,188.0
3,Adams E.,USA,1959.0,73.0,178.0
4,Agenor R.,USA,1964.0,82.0,180.0
...,...,...,...,...,...
10907,Zorab A.,UNK,1976.0,77.0,183.0
10908,Zimmermann J.,UNK,1976.0,77.0,183.0
10909,Zinn G.,UNK,1976.0,77.0,183.0
10910,Zenger C.,GER,1976.0,77.0,183.0


In [61]:
# Merge atp players with our main dataset 
merged_df = pd.merge(merged_df, winner_players_df, on=['Winner'])
merged_df = pd.merge(merged_df, loser_players_df, on=['Loser'])
merged_df

,Date,Series,Court,Surface,Winner,Loser,Wsets_percent,W_round_pts_avg_percent,Month,Day,...,Сloudcover,Shortwave_radiation,W_flag_code,W_birth_year,W_weight_kg,W_height_cm,L_flag_code,L_birth_year,L_weight_kg,L_height_cm
0,2000-01-03,International,Outdoor,Hard,Dosedel S.,Ljubicic I.,1.000000,0.666667,1,3,...,40.500000,0.000000,CZE,1970.0,79.0,183.0,CRO,1979.0,92.0,193.0
1,2000-05-01,International,Outdoor,Clay,Enqvist T.,Ljubicic I.,0.666667,0.558824,5,1,...,39.000000,579.333333,SWE,1974.0,88.0,191.0,CRO,1979.0,92.0,193.0
2,2001-02-19,International Gold,Indoor,Hard,Escude N.,Ljubicic I.,1.000000,0.750000,2,19,...,100.000000,0.000000,FRA,1976.0,70.0,185.0,CRO,1979.0,92.0,193.0
3,2002-09-30,International,Indoor,Carpet,Escude N.,Ljubicic I.,1.000000,0.565217,9,30,...,100.000000,0.000000,FRA,1976.0,70.0,185.0,CRO,1979.0,92.0,193.0
4,2004-01-10,International,Outdoor,Hard,Escude N.,Ljubicic I.,1.000000,0.590909,1,10,...,25.333333,196.000000,FRA,1976.0,70.0,185.0,CRO,1979.0,92.0,193.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54718,2016-07-12,ATP250,Outdoor,Clay,Soderlund C.,Simonsson F.,0.666667,0.516129,7,12,...,36.833333,525.833333,SWE,1997.0,80.0,188.0,SWE,1994.0,75.0,188.0
54719,2021-11-10,ATP250,Indoor,Hard,Shapovalov D.,Vavassori A.,1.000000,0.650000,11,10,...,100.000000,0.000000,CAN,1999.0,76.0,183.0,ITA,1995.0,78.0,193.0
54720,2021-07-28,ATP250,Outdoor,Clay,Ymer M.,Erler A.,1.000000,0.705882,7,28,...,93.833333,299.333333,SWE,1998.0,75.0,183.0,AUT,1997.0,77.0,183.0
54721,2017-08-01,ATP250,Outdoor,Hard,King E.,Sanchez M.,1.000000,0.600000,8,1,...,92.333333,434.000000,USA,1992.0,79.0,185.0,MEX,1991.0,76.0,183.0


Once we make small adjustments to our data, it will be ready for encoding.

In [62]:
# Define function to calculate age for each row
def calculate_age(row, player):
    if (player == "winner"):
        birth_year = row['W_birth_year']
    else:
        birth_year = row['L_birth_year']
    game_date = row['Date'].strftime('%Y-%m-%d')
    game_datetime = datetime.strptime(game_date, '%Y-%m-%d')
    player_age = game_datetime.year - birth_year
      
    return player_age

# Calulate age for each player
merged_df['W_age'] = merged_df.apply(lambda row: calculate_age(row, "winner"), axis=1)
merged_df['L_age'] = merged_df.apply(lambda row: calculate_age(row, "loser"), axis=1)

In [64]:
# Create function for calculating the head-to-head ratio for past matches
def calculate_h2h_winning_ratio(data: pd.DataFrame, date: pd.Series, player_1: pd.Series, player_2: pd.Series) -> float:
    past_matches = data[(data['Date'] < date) &
                        (((data['Winner'] == player_1) & (data['Loser'] == player_2)) |
                         ((data['Winner'] == player_2) & (data['Loser'] == player_1)))]

    if len(past_matches) == 0:
        return 0.5

    player_1_wins = (past_matches['Winner'] == player_1).sum()
    player_1_winning_ratio = player_1_wins / len(past_matches)

    return player_1_winning_ratio

# Randomly map Winner and Loser to player_1 and player_2
merged_df['player_1'] = np.where(np.random.rand(len(merged_df)) > 0.5, merged_df['Winner'], merged_df['Loser'])
merged_df['player_2'] = np.where(merged_df['player_1'] == merged_df['Winner'], merged_df['Loser'], merged_df['Winner'])

# Create target variable: 1 if player_1 is the winner, 0 otherwise
merged_df['player_1_win'] = (merged_df['player_1'] == merged_df['Winner']).astype(int)

# Apply the head to head ratio and drop winner and loser columns
merged_df['player_1_h2h_winning_ratio'] = merged_df.apply(lambda row: calculate_h2h_winning_ratio(merged_df, row['Date'], row['player_1'], row['player_2']), axis=1)

merged_df = merged_df.drop(merged_df[["Winner", "Loser", "Date", "W_birth_year", "L_birth_year"]], axis = 1)
merged_df

,Series,Court,Surface,Wsets_percent,W_round_pts_avg_percent,Month,Day,Year,W_over_L_rank_difference_normalized,W_played_here_before,...,W_height_cm,L_flag_code,L_weight_kg,L_height_cm,W_age,L_age,player_1,player_2,player_1_win,player_1_h2h_winning_ratio
0,International,Outdoor,Hard,1.000000,0.666667,1,3,2000,0.200671,False,...,183.0,CRO,92.0,193.0,30.0,21.0,Dosedel S.,Ljubicic I.,1,0.5
1,International,Outdoor,Clay,0.666667,0.558824,5,1,2000,2.257123,False,...,191.0,CRO,92.0,193.0,26.0,21.0,Enqvist T.,Ljubicic I.,1,0.5
2,International Gold,Indoor,Hard,1.000000,0.750000,2,19,2001,0.427444,True,...,185.0,CRO,92.0,193.0,25.0,22.0,Ljubicic I.,Escude N.,0,0.5
3,International,Indoor,Carpet,1.000000,0.565217,9,30,2002,-0.260283,True,...,185.0,CRO,92.0,193.0,26.0,23.0,Escude N.,Ljubicic I.,1,1.0
4,International,Outdoor,Hard,1.000000,0.590909,1,10,2004,-0.998529,True,...,185.0,CRO,92.0,193.0,28.0,25.0,Ljubicic I.,Escude N.,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54718,ATP250,Outdoor,Clay,0.666667,0.516129,7,12,2016,0.341530,False,...,188.0,SWE,75.0,188.0,19.0,22.0,Soderlund C.,Simonsson F.,1,0.5
54719,ATP250,Indoor,Hard,1.000000,0.650000,11,10,2021,2.776055,True,...,183.0,ITA,78.0,193.0,22.0,26.0,Vavassori A.,Shapovalov D.,0,0.5
54720,ATP250,Outdoor,Clay,1.000000,0.705882,7,28,2021,1.224963,True,...,183.0,AUT,77.0,183.0,23.0,24.0,Erler A.,Ymer M.,0,0.5
54721,ATP250,Outdoor,Hard,1.000000,0.600000,8,1,2017,0.586826,False,...,185.0,MEX,76.0,183.0,25.0,26.0,Sanchez M.,King E.,0,0.5


In [3]:
# Use One-Hot encoder to encode categorial values with a low level of uniqueness
ohe_encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
features = ['Series', 'Court', 'Surface']
ohe_encoder.fit(merged_df[features])

new_feats = ohe_encoder.transform(merged_df[features])
new_cols = pd.DataFrame(new_feats, dtype=int, columns=ohe_encoder.get_feature_names(features))
merged_df = pd.concat([merged_df, new_cols], axis=1)    
merged_df.drop(features, axis=1, inplace=True)

merged_df

C:\Users\markz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Wsets_percent,W_round_pts_avg_percent,Month,Day,Year,W_over_L_rank_difference_normalized,W_played_here_before,L_played_here_before,Temperature,Windspeed_10m,...,Series_International Gold,Series_Masters,Series_Masters 1000,Series_Masters Cup,Court_Indoor,Court_Outdoor,Surface_Carpet,Surface_Clay,Surface_Grass,Surface_Hard
0,1.000000,0.666667,1,3,2000,0.200671,False,False,15.050000,27.750000,...,0,0,0,0,0,1,0,0,0,1
1,0.666667,0.558824,5,1,2000,2.257123,False,False,18.966667,8.766667,...,0,0,0,0,0,1,0,1,0,0
2,1.000000,0.750000,2,19,2001,0.427444,True,False,23.000000,0.000000,...,1,0,0,0,1,0,0,0,0,1
3,1.000000,0.565217,9,30,2002,-0.260283,True,False,23.000000,0.000000,...,0,0,0,0,1,0,1,0,0,0
4,1.000000,0.590909,1,10,2004,-0.998529,True,True,23.983333,21.516667,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54718,0.666667,0.516129,7,12,2016,0.341530,False,False,18.250000,24.433333,...,0,0,0,0,0,1,0,1,0,0
54719,1.000000,0.650000,11,10,2021,2.776055,True,True,23.000000,0.000000,...,0,0,0,0,1,0,0,0,0,1
54720,1.000000,0.705882,7,28,2021,1.224963,True,True,18.183333,9.516667,...,0,0,0,0,0,1,0,1,0,0
54721,1.000000,0.600000,8,1,2017,0.586826,False,False,18.916667,9.750000,...,0,0,0,0,0,1,0,0,0,1


In [8]:
# Make lists for encoding 
unique_flags = list(set(list(merged_df['W_flag_code'].unique()) + list(merged_df['L_flag_code'].unique())))
encoder_flags = [i for i in range(80)]
unique_players = list(set(list(merged_df['player_1'].unique()) + list(merged_df['player_2'].unique())))
encoder_players = [i for i in range(1178)]

# Create dictionaries for encoding 'W_flag_code', 'L_flag_code', 'Winner', and 'Loser' columns
flag_dict = dict(zip(unique_flags, encoder_flags))
player_dict = dict(zip(unique_players, encoder_players))

# Use encoding dictionaries to encode 'W_flag_code', 'L_flag_code', 'player_1', and 'player_2' columns
merged_df['W_flag_code'] = merged_df['W_flag_code'].map(flag_dict)
merged_df['L_flag_code'] = merged_df['L_flag_code'].map(flag_dict)
merged_df['player_1'] = merged_df['player_1'].map(player_dict)
merged_df['player_2'] = merged_df['player_2'].map(player_dict)

merged_df

,Wsets_percent,W_round_pts_avg_percent,Month,Day,Year,W_over_L_rank_difference_normalized,W_played_here_before,L_played_here_before,Temperature,Windspeed_10m,...,Series_International Gold,Series_Masters,Series_Masters 1000,Series_Masters Cup,Court_Indoor,Court_Outdoor,Surface_Carpet,Surface_Clay,Surface_Grass,Surface_Hard
0,1.000000,0.666667,1,3,2000,0.200671,False,False,15.050000,27.750000,...,0,0,0,0,0,1,0,0,0,1
1,0.666667,0.558824,5,1,2000,2.257123,False,False,18.966667,8.766667,...,0,0,0,0,0,1,0,1,0,0
2,1.000000,0.750000,2,19,2001,0.427444,True,False,23.000000,0.000000,...,1,0,0,0,1,0,0,0,0,1
3,1.000000,0.565217,9,30,2002,-0.260283,True,False,23.000000,0.000000,...,0,0,0,0,1,0,1,0,0,0
4,1.000000,0.590909,1,10,2004,-0.998529,True,True,23.983333,21.516667,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54718,0.666667,0.516129,7,12,2016,0.341530,False,False,18.250000,24.433333,...,0,0,0,0,0,1,0,1,0,0
54719,1.000000,0.650000,11,10,2021,2.776055,True,True,23.000000,0.000000,...,0,0,0,0,1,0,0,0,0,1
54720,1.000000,0.705882,7,28,2021,1.224963,True,True,18.183333,9.516667,...,0,0,0,0,0,1,0,1,0,0
54721,1.000000,0.600000,8,1,2017,0.586826,False,False,18.916667,9.750000,...,0,0,0,0,0,1,0,0,0,1


In [10]:
merged_df.to_excel('final_dataset.xlsx', index=False)